In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

# Print assigned GPU
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

# Set TensorFlow version to 1.x
%tensorflow_version 1.x

# Change dir to project dir
%cd /content/gdrive/MyDrive/Colab\ Data/COCO-Human-Pose

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Fri Mar 12 00:33:51 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    23W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----

In [ ]:
# Install colab_ssh on google colab
#!pip install colab_ssh --upgrade

#from colab_ssh import launch_ssh_cloudflared
#launch_ssh_cloudflared(password="")


In [2]:
# (<5 mins)
# Setup train/val symlinks for this runtime
# puts them in /content/datasets/coco
# ensure you have added a Drive shortcut to the dataset in the root of your Drive
# this is necessary as the symlinks need a destination that exists
# the dataset is hosted here: https://drive.google.com/drive/folders/1EVsLBRwT2njNWOrmBAhDHvvB8qrd9pXT 
import time
from datetime import datetime, timedelta

start = time.time()

!unzip /content/gdrive/MyDrive/COCO/2017/train2017_symlinks-1000.zip -d -q /content
!unzip /content/gdrive/MyDrive/COCO/2017/val2017_symlinks-1000.zip -d -q /content

end = time.time()

setup_time = end - start

print("Total setup time: {}".format(str(timedelta(seconds=setup_time))))

Streaming output truncated to the last 5000 lines.
  /content/datasets/coco/val2017/000000535306.jpg -> /content/gdrive/My Drive/COCO/2017/val2017_sub-1000/4/000000535306.jpg
  /content/datasets/coco/val2017/000000080671.jpg -> /content/gdrive/My Drive/COCO/2017/val2017_sub-1000/0/000000080671.jpg
  /content/datasets/coco/val2017/000000314541.jpg -> /content/gdrive/My Drive/COCO/2017/val2017_sub-1000/2/000000314541.jpg
  /content/datasets/coco/val2017/000000215778.jpg -> /content/gdrive/My Drive/COCO/2017/val2017_sub-1000/1/000000215778.jpg
  /content/datasets/coco/val2017/000000384670.jpg -> /content/gdrive/My Drive/COCO/2017/val2017_sub-1000/3/000000384670.jpg
  /content/datasets/coco/val2017/000000389381.jpg -> /content/gdrive/My Drive/COCO/2017/val2017_sub-1000/3/000000389381.jpg
  /content/datasets/coco/val2017/000000179174.jpg -> /content/gdrive/My Drive/COCO/2017/val2017_sub-1000/1/000000179174.jpg
  /content/datasets/coco/val2017/000000370270.jpg -> /content/gdrive/My Drive/COC

In [10]:
!ls /content/datasets/coco/train2017 | wc -l
!ls /content/datasets/coco/val2017 | wc -l

118324
5000


In [18]:
# (10-20 mins)
# Populate the runtimes filesystem cache with dest directories for fast and predictable load times
start = time.time()

import os
def print_files_in_dir(path):
    print(len(os.listdir(path)), " files at ", path)

for i in range(0,6):
    print_files_in_dir('/content/gdrive/MyDrive/COCO/2017/val2017_sub-1000/' + str(i))

for i in range(0,119):
    print_files_in_dir('/content/gdrive/MyDrive/COCO/2017/train2017_sub-1000/' + str(i))
  
end = time.time()

setup_time = end - start

print("Total setup time: {}".format(str(timedelta(seconds=setup_time))))

999  files at  /content/gdrive/MyDrive/COCO/2017/val2017_sub-1000/0
999  files at  /content/gdrive/MyDrive/COCO/2017/val2017_sub-1000/1
999  files at  /content/gdrive/MyDrive/COCO/2017/val2017_sub-1000/2
999  files at  /content/gdrive/MyDrive/COCO/2017/val2017_sub-1000/3
999  files at  /content/gdrive/MyDrive/COCO/2017/val2017_sub-1000/4
5  files at  /content/gdrive/MyDrive/COCO/2017/val2017_sub-1000/5
999  files at  /content/gdrive/MyDrive/COCO/2017/train2017_sub-1000/0
999  files at  /content/gdrive/MyDrive/COCO/2017/train2017_sub-1000/1
999  files at  /content/gdrive/MyDrive/COCO/2017/train2017_sub-1000/2
999  files at  /content/gdrive/MyDrive/COCO/2017/train2017_sub-1000/3
999  files at  /content/gdrive/MyDrive/COCO/2017/train2017_sub-1000/4
999  files at  /content/gdrive/MyDrive/COCO/2017/train2017_sub-1000/5
999  files at  /content/gdrive/MyDrive/COCO/2017/train2017_sub-1000/6
999  files at  /content/gdrive/MyDrive/COCO/2017/train2017_sub-1000/7
999  files at  /content/gdrive/MyD

In [9]:
# Example Usage of Data Generator
from constants import INPUT_DIM
from constants import OUTPUT_DIM
from data_generator import DataGenerator

train_gen = DataGenerator("data/one_ann_min10kp_imgs.csv",
                          "/content/datasets/coco/",
                          INPUT_DIM,
                          OUTPUT_DIM,
                          num_hg_blocks=8,
                          batch_size=1,
                          shuffle=True,
                          online_fetch=True)
# now let's train the model
#model.fit(train_gen, epochs=5, ...)

In [7]:
!git submodule update --init --recursive

Submodule 'submodules/HeatMap' (https://github.com/LinShanify/HeatMap) registered for path 'submodules/HeatMap'
Cloning into '/content/gdrive/My Drive/Colab Data/COCO-Human-Pose/submodules/HeatMap'...
Submodule path 'submodules/HeatMap': checked out 'b1cb74cd7c3a209d285b5af4554b385c74e0eac0'


In [15]:
!git checkout robert/training-setup



M	notebooks/coco-explorer.ipynb
M	notebooks/input_pipeline_demo.ipynb
Already on 'robert/training-setup'
Your branch is up to date with 'origin/robert/training-setup'.
TensorFlow is already loaded. Please restart the runtime to change versions.


In [17]:
!python3 train.py --epochs 100 --batch 24 --hourglass 1

Using TensorFlow backend.


Setup start: Fri Mar 12 01:16:49 2021

TensorFlow detected the following GPU(s):
2021-03-12 01:16:49.439426: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2199995000 Hz
2021-03-12 01:16:49.439625: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55e4b5700840 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-03-12 01:16:49.439656: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-03-12 01:16:49.441344: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-03-12 01:16:49.546590: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-03-12 01:16:49.547498: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5

In [10]:
# Test the generator
import time
import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (10.0, 10.0)

from submodules.HeatMap import HeatMap # https://github.com/LinShanify/HeatMap

start = time.time()
X_batch, y_batch = train_gen[15]
X,y = X_batch[0], y_batch[0] # take first example of batch
print("Retrieving batch took: ",time.time() - start, " s")
print(X.shape,y.shape)

for i in range(NUM_COCO_KEYPOINTS):
  hm = HeatMap(X,y[:,:,i])
  hm.plot(transparency=0.5,show_axis=True,show_colorbar=True)

AttributeError: ignored

In [ ]:
for i in range(17):
  X, y = X_batch[i], y_batch[i]
  xs = [v for i,v in enumerate(y) if i%3==0]
  ys = [v for i,v in enumerate(y) if i%3==1]
  plt.subplot(5,5,i+1)
  plt.imshow(X)
  plt.scatter(xs,ys)